In [187]:
import sys
import torch
from torch import nn
import torch.nn.functional as F
from math import exp
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plot

# 1. ***用于二分类任务或重建任务，loss function bce***
***F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)***
- weight (Tensor, optional): a manual rescaling weight given to the loss of each batch element.
- This is used for measuring the error of a reconstruction in for example an auto-encoder.

***The unreduced (i.e. with `reduction` set to ``'none'``) loss can be described as***

$$
\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
l_n = - w_n \left[ y_n \cdot \log x_n + (1 - y_n) \cdot \log (1 - x_n) \right],
$$

- `N` is the batch size. 
- targets `y` should be numbers between 0 and 1.

***If `reduction` is not ``'none'`` (default ``'mean'``), then***

$$
\ell(x, y) = \begin{cases}
\operatorname{mean}(L), \quad & \text{if reduction} = \text{`mean';}\\
\operatorname{sum}(L), \quad & \text{if reduction} = \text{`sum'.}
\end{cases}
$$

In [168]:
bce = nn.BCELoss()

def binary_cross_entorpy(inputs, targets):
    inputs = inputs.numpy()
    inputs = inputs.reshape((inputs.shape[0]*inputs.shape[1], inputs.shape[-2]*inputs.shape[-1]))
    targets = targets.numpy()
    targets = targets.reshape((targets.shape[0]*targets.shape[1], targets.shape[-2]*targets.shape[-1]))
    outputs = 0.
    weight = 1.
    for i in range(targets.shape[0]):
        temp = 0
        for j in range(targets.shape[1]):
            temp += -1. * weight * (targets[i, j]*np.log(inputs[i, j]) + (1-targets[i, j])*np.log(1-inputs[i, j]))
        outputs += (temp / targets.shape[1])
        
    return outputs / targets.shape[0]

In [250]:
inputs = torch.rand((1, 2, 2, 2))
outputs = torch.tensor([[[[0, 1.], [1., 0]], [[0, 1.], [1., 0]]]])
# outputs = torch.nn.Softmax(dim=1)(torch.rand(1, 2, 2, 2))

print(inputs,
      outputs,
      f'bce {bce(inputs, outputs):6f}',
      f'binary_cross_entorpy {binary_cross_entorpy(inputs, outputs):6f}',
      sep="\n")

tensor([[[[0.4346, 0.9757],
          [0.5365, 0.6144]],

         [[0.2446, 0.2691],
          [0.7291, 0.4737]]]])
tensor([[[[0.5867, 0.4917],
          [0.5620, 0.5491]],

         [[0.4133, 0.5083],
          [0.4380, 0.4509]]]])
bce 0.892566
binary_cross_entorpy 0.892566


# 2. ***用于多分类任务，loss function ce***
***F.cross_entropy(input, target, weight=self.weight, ignore_index=self.ignore_index, reduction=self.reduction, label_smoothing=self.label_smoothing)***
- this case is equivalent to the combination of `~torch.nn.LogSoftmax` and `~torch.nn.NLLLoss`.
  > $$\text{LogSoftmax}(x_{i}) = \log\left(\frac{\exp(x_i) }{ \sum_j \exp(x_j)} \right)$$
  > $$Nll(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
        l_n = - w_{y_n} x_{n,y_n}, \quad
        w_{c} = \text{weight}[c] \cdot \mathbb{1}\{c \not= \text{ignore\_index}\}$$
- It is useful when training a classification problem with `C` classes.
- If provided, the optional argument `weight` should be a 1D `Tensor` assigning weight to each of the classes, This is
particularly useful when you have an unbalanced training set.

***The `input` is expected to contain raw, unnormalized scores for each class.***
- `input` has to be a Tensor of size `(C)` for unbatched input, `(N, C)` or `(N, C, d_1, d_2, ..., d_K)` with $K\geq 1$ for the `K`-dimensional case.
  > $$\begin{aligned}
      C ={} & \text{number of classes} \\
      N ={} & \text{batch size} \\
      \end{aligned}$$

***The `target` that this criterion expects should contain either***
- Class indices in the range `[0, C)` where `C` is the number of classes, not one-hot, dtype is long.
- if `ignore_index` is specified, loss also accepts this class index (this index may not necessarily be in the class range).
- If containing class probabilities, same shape as the input and each value should be between `[0, 1]`, dtype is float.
- The unreduced (i.e. with `reduction` set to ``'none'``) loss for this case can be described as

$$
\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
l_n = - w_{y_n} \log \frac{\exp(x_{n,y_n})}{\sum_{c=1}^C \exp(x_{n,c})}
\cdot \mathbb{1}\{y_n \not= \text{ignore\_index}\}
$$

`x` is the input, `y` is the target, `w` is the weight,
`C` is the number of classes, and `N` spans the minibatch dimension as well as `d_1, ..., d_k` for the `K`-dimensional case.

- The performance of this criterion is generally better when `target` contains class
  indices, as this allows for optimized computation. Consider providing `target` as
  class probabilities only when a single class label per minibatch item is too restrictive.

***The `output` If reduction is 'none', same shape as the target. Otherwise, scalar.***

In [252]:
ce = nn.CrossEntropyLoss()
ls = nn.LogSoftmax(dim=1)
nll = nn.NLLLoss()


def cross_entorpy(inputs, targets):
    inputs = inputs.numpy()
    targets = targets.numpy()
    outputs = 0.
    weight = 1.
    if targets.dtype == np.int64:
        assert len(inputs.shape) == 4 and len(targets.shape) == 3
        for k in range(targets.shape[0]):
            temp = 0.
            for i in range(targets.shape[-2]):
                for j in range(targets.shape[-1]):
                    temp += -1. * weight * (np.log(np.exp(inputs[k, :, i, j][..., int(targets[k, i, j].item())]) /
                            np.sum(np.exp(inputs[k, :, i, j]))))
            outputs += temp
    elif targets.dtype == np.float32:
        assert inputs.shape == targets.shape
        for k in range(targets.shape[0]):
            temp = 0.
            for i in range(targets.shape[-2]):
                for j in range(targets.shape[-1]):
                    temp += -1. * weight * np.sum(np.log(np.exp(inputs[k, :, i, j]) / np.sum(np.exp(inputs[k, :, i, j]))) * targets[k, :, i, j])
            outputs += temp
    else:
        print(f'标签的数据类型应该是 int64 或者 float32 而不是 {targets.dtype}')
        sys.exit()

    return (outputs / (targets.shape[0] * targets.shape[-2] * targets.shape[-1])).item()

In [254]:
# 交叉熵的计算模式 - 标签中的元素是类的索引值, [0, C-1] -> int64
inputs = torch.rand(2, 2, 5, 5)
targets = torch.rand(2, 5, 5).random_(2).long()

# # 交叉熵的计算模式 - 标签中的元素是类的概率值, [0, 1] -> float32
# inputs = torch.rand(2, 2, 5, 5)
# targets = torch.nn.Softmax(dim=1)(torch.rand(2, 2, 5, 5))

outputs = ce(inputs, targets)
print(f'ce {outputs:6f}')

if targets.dtype == torch.int64:
    outputs = nll(ls(inputs), targets)
    print(f'logsoftmax+nll {outputs:6f}')

outputs = cross_entorpy(inputs, targets)
print(f'cross_entorpy {outputs:6f}')

ce 0.725609
logsoftmax+nll 0.725609
cross_entorpy 0.725609


# 3. ***用于重建任务，loss function ssim***
- 结构相似性指数（structural similarity index，SSIM）用于度量两幅图像间的结构相似性。和被广泛采用的L2 loss不同，SSIM和人类的视觉系统（HVS）类似，对局部结构变化的感知敏感

In [ ]:
# 计算一维的高斯分布向量
def gaussian(window_size, sigma):
    gauss = torch.Tensor([
        exp(-(x - window_size // 2)**2 / float(2 * sigma**2))
        for x in range(window_size)
    ])
    return gauss / gauss.sum()


# 创建高斯核，通过两个一维高斯分布向量进行矩阵乘法得到
# 可以设定channel参数拓展为3通道
def create_window(window_size, channel=1):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(
        _1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = _2D_window.expand(channel, 1, window_size,
                               window_size).contiguous()
    return window


# 计算SSIM
# 直接使用SSIM的公式，但是在计算均值时，不是直接求像素平均值，而是采用归一化的高斯核卷积来代替。
# 在计算方差和协方差时用到了公式Var(X)=E[X^2]-E[X]^2, cov(X,Y)=E[XY]-E[X]E[Y].
# 正如前面提到的，上面求期望的操作采用高斯核卷积代替。
def ssim(img1,
         img2,
         window_size=11,
         window=None,
         size_average=True,
         full=False,
         val_range=None):
    # Value range can be different from 255. Other common ranges are 1 (sigmoid) and 2 (tanh).
    if val_range is None:
        if torch.max(img1) > 128:
            max_val = 255
        else:
            max_val = 1

        if torch.min(img1) < -0.5:
            min_val = -1
        else:
            min_val = 0
        L = max_val - min_val
    else:
        L = val_range

    padd = 0
    (_, channel, height, width) = img1.size()
    if window is None:
        real_size = min(window_size, height, width)
        window = create_window(real_size, channel=channel).to(img1.device)

    mu1 = F.conv2d(img1, window, padding=padd, groups=channel)
    mu2 = F.conv2d(img2, window, padding=padd, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=padd,
                         groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=padd,
                         groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=padd,
                       groups=channel) - mu1_mu2

    C1 = (0.01 * L)**2
    C2 = (0.03 * L)**2

    v1 = 2.0 * sigma12 + C2
    v2 = sigma1_sq + sigma2_sq + C2
    cs = torch.mean(v1 / v2)  # contrast sensitivity

    ssim_map = ((2 * mu1_mu2 + C1) * v1) / ((mu1_sq + mu2_sq + C1) * v2)

    if size_average:
        ret = ssim_map.mean()
    else:
        ret = ssim_map.mean(1).mean(1).mean(1)

    if full:
        return ret, cs
    return ret


# Classes to re-use window
class SSIM(torch.nn.Module):

    def __init__(self, window_size=11, size_average=True, val_range=None):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average
        self.val_range = val_range

        # Assume 1 channel for SSIM
        self.channel = 1
        self.window = create_window(window_size)

    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()

        if channel == self.channel and self.window.dtype == img1.dtype:
            window = self.window
        else:
            window = create_window(self.window_size,
                                   channel).to(img1.device).type(img1.dtype)
            self.window = window
            self.channel = channel

        return ssim(img1,
                    img2,
                    window=window,
                    window_size=self.window_size,
                    size_average=self.size_average)
